In [ ]:
#IE407 Internet of Things Project Work
#Names: Visaj Shah, Aayush Desai
#IDs: 201801016, 201801060

**Project: Analyzing Water Suitability for Aquaculture**

**Notebook 3: AWS + KNN Implementation**

In [ ]:
#This notebook is similar to Notebook 2 except that the dataset is directly imported from AWS server every time it is run.
#This means the Training set and Testing set also keep changing every time.

In [ ]:
#Importing the required libraries

import sys
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from AWSIoTPythonSDK.MQTTLib import AWSIoTMQTTClient
import boto3

In [ ]:
#User's input

dictIn = json.loads(sys.argv[1])

ph = float(dictIn['ph'])
temp30 = float(dictIn['temp1'])
turb30 = int(dictIn['tur1'])
temp60 = float(dictIn['temp2'])
turb60 = int(dictIn['tur2'])

userIn = [[ph, temp30, turb30, temp60, turb60]]

dictAWS = {}

for i in dictIn.keys():
  if i == 'ph':
    dictAWS['ph'] = ph
  elif i == 'temp1':
    dictAWS['temp30'] = temp30
  elif i == 'tur1':
    dictAWS['turb30'] = turb30
  elif i == 'temp2':
    dictAWS['temp60'] = temp60
  else:
    dictAWS['turb60'] = turb60

In [ ]:
#Connecting with the MQTT Client (on AWS)

myMQTTClient = AWSIoTMQTTClient("lab1_thing")

myMQTTClient.configureEndpoint("", )
myMQTTClient.configureCredentials("")

myMQTTClient.connect()

In [ ]:
#Creating the model

client1 = boto3.client(
    'iotanalytics',
    region_name='us-east-2',
    aws_access_key_id='',
    aws_secret_access_key=''
)
dataset = "project_dst"
dataset_url = client1.get_dataset_content(datasetName = dataset)['entries'][0]['dataURI']

dataset_url = ""

data = pd.read_csv(dataset_url)

x = df[['ph', 'temp30', 'turb30', 'temp60', 'turb60']]
y = df['class']

xTrainSet, xTestSet, yTrainSet, yTestSet = train_test_split(x, y, test_size = 0.3)

knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(xTrainSet, yTrainSet)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

In [ ]:
#Predicting for userIn

yPred = knn.predict(userIn)
print(yPred[0])

1


In [ ]:
#Sending User Input and Model Output to Cloud (AWS)

dictAWS['class'] = yPred[0]
msg = json.dumps(dictAWS)
topic = "update/environment/dht1"
myMQTTClient.publish(topic, msg, 0) 

myMQTTClient.disconnect()